# Exploring Keyboard Data

1. Load the data
    - Create a dataframe of all data folders
    - additional features are Name of user, filename, Keyboard Material, microphone
2. Clean up where it's unknown
    - Impute values from online, or set generic value.


End result: One dataframe of information

Cluster data

Visualize those clusters

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import yaml

In [3]:
# Define the path to the data folder
data_folder = '../data_copy/'

# Initialize an empty list to store the data
key_data_list = []
sentence_data_list = []

# Loop through all user folders in the data folder
for user_folder in os.listdir(data_folder):
    if 'Keystroke' in user_folder:
        continue
    user_folder_path = os.path.join(data_folder, user_folder)
    if os.path.isdir(user_folder_path):  # Check if it's a directory
        key_folder_path = os.path.join(user_folder_path, 'data', 'key_data')
        sentence_folder_path = os.path.join(user_folder_path, 'data', 'sentence_data')

        # Key data, then sentence data
        # for internal_folder in [key_folder_path, sentence_folder_path]:
        for file_name in os.listdir(key_folder_path):
            # key_folder_path = os.path.join(data_folder, user_folder)
            # Check if the file is a YAML file
            if file_name.endswith('.yaml'):
                # Extract key_pressed value from the filename
                key_pressed = file_name.split('_')[2]
                
                # Construct the full path to the YAML file
                yaml_file_path = os.path.join(key_folder_path, file_name)
                
                # Read the YAML file and filter out the 'key_pressed' field and subsequent lines until 'keyboard_name'
                with open(yaml_file_path, 'r') as file:
                    lines = file.readlines()
                
                filtered_lines = []
                skip = False
                for line in lines:
                    if line.strip().startswith('key_pressed:'):
                        skip = True
                    if skip and line.strip().startswith('keyboard_name:'):
                        skip = False
                    if not skip:
                        filtered_lines.append(line)
                
                # Load the filtered lines into a dictionary
                yaml_data = yaml.safe_load(''.join(filtered_lines))
                
                # Add the user, audio file name, and key_pressed information
                yaml_data['user'] = user_folder
                yaml_data['audio_file'] = file_name.replace('.yaml', '.wav')  # Assuming audio files are in .wav format
                yaml_data['key_pressed'] = key_pressed
                
                # Append the data to the list
                key_data_list.append(yaml_data)

# Convert the list of dictionaries to a DataFrame
key_df = pd.DataFrame(key_data_list)
sentence_df = pd.DataFrame(sentence_data_list)

# Display the DataFrame
key_df.head()

,default_high_input_latency,default_high_output_latency,default_low_input_latency,default_low_output_latency,default_samplerate,hostapi,index,keyboard_name,keyboard_size,keyboard_type,max_input_channels,max_output_channels,name,switch_color,timestamp,user,audio_file,key_pressed
0,0.18,0.18,0.09,0.09,44100.0,0,1,DropCTRLV1,80%_Tenkeyless,mechanical,2,0,Mic/Inst (Samson G-Track Pro),Halo True,1720383683,Basem,key_press_x_1720383683_Basem.wav,x
1,0.18,0.18,0.09,0.09,44100.0,0,1,DropCTRLV1,80%_Tenkeyless,mechanical,2,0,Mic/Inst (Samson G-Track Pro),Halo True,1720382929,Basem,key_press_]_1720382929_Basem.wav,]
2,0.18,0.18,0.09,0.09,44100.0,0,1,DropCTRLV1,80%_Tenkeyless,mechanical,2,0,Mic/Inst (Samson G-Track Pro),Halo True,1720383217,Basem,key_press_g_1720383217_Basem.wav,g
3,0.18,0.18,0.09,0.09,44100.0,0,1,DropCTRLV1,80%_Tenkeyless,mechanical,2,0,Mic/Inst (Samson G-Track Pro),Halo True,1720380641,Basem,key_press_1_1720380641_Basem.wav,1
4,0.18,0.18,0.09,0.09,44100.0,0,1,DropCTRLV1,80%_Tenkeyless,mechanical,2,0,Mic/Inst (Samson G-Track Pro),Halo True,1720380770,Basem,key_press_3_1720380770_Basem.wav,3


## Add Keyboard Material Column

In [ ]:
key_df['keyboard_name'].unique()

material_mapper = {'DropCTRLV1': 'aluminum',
                   'Keychron Q3': 'aluminum'}

key_df['keyboard_material'] = key_df['keyboard_name'].map(material_mapper)
key_df.drop('keyboard_name', axis=1, inplace=True)

## Add Keyboard Microhone Column cleanup

In [ ]:
key_df['name'].unique()

microphone_mapper = {'Mic/Inst (Samson G-Track Pro)': 'Samson_GTrack_Pro',
                     'Microphone (3- USB PnP Audio De': 'GenericPnP'}

key_df['microphone'] = key_df['name'].map(microphone_mapper)
key_df.drop('name', axis=1, inplace=True)

## Drop unneeded columns

In [ ]:
unnecessary_cols = ['default_high_input_latency',
                    'default_high_output_latency',
                    'default_low_input_latency',
                    'default_low_output_latency',
                    'default_samplerate',
                    'hostapi',
                    'index',
                    'max_input_channels',
                    'max_output_channels',
                    'timestamp',]
key_df.drop(unnecessary_cols, axis=1, inplace=True)

In [ ]:
key_df[key_df['key_pressed'] == 'p']

,keyboard_size,keyboard_type,switch_color,user,audio_file,key_pressed,keyboard_material,microphone
330,80%_Tenkeyless,mechanical,Halo True,Basem,key_press_p_1720382796.wav,p,aluminum,Samson_GTrack_Pro
331,80%_Tenkeyless,mechanical,Halo True,Basem,key_press_p_1720382799.wav,p,aluminum,Samson_GTrack_Pro
332,80%_Tenkeyless,mechanical,Halo True,Basem,key_press_p_1720382803.wav,p,aluminum,Samson_GTrack_Pro
333,80%_Tenkeyless,mechanical,Halo True,Basem,key_press_p_1720382806.wav,p,aluminum,Samson_GTrack_Pro
334,80%_Tenkeyless,mechanical,Halo True,Basem,key_press_p_1720382810.wav,p,aluminum,Samson_GTrack_Pro
335,80%_Tenkeyless,mechanical,Halo True,Basem,key_press_p_1720382813.wav,p,aluminum,Samson_GTrack_Pro
336,80%_Tenkeyless,mechanical,Halo True,Basem,key_press_p_1720382817.wav,p,aluminum,Samson_GTrack_Pro
337,80%_Tenkeyless,mechanical,Halo True,Basem,key_press_p_1720382820.wav,p,aluminum,Samson_GTrack_Pro
338,80%_Tenkeyless,mechanical,Halo True,Basem,key_press_p_1720382823.wav,p,aluminum,Samson_GTrack_Pro
339,80%_Tenkeyless,mechanical,Halo True,Basem,key_press_p_1720382827.wav,p,aluminum,Samson_GTrack_Pro


TODO:
- Add a name abbreviation for each audio file for the respective folders
- convert Switches to clicky or non clicky
    - Clicky: 
        - Bumpy
        - loud noise
        - Blue, green
    - Tactile:
        Bumpy, moderate noise. You feel the bump before it actuates. Usually no sound (halo clears, mx clears)
        - Brown, clear
    - Linear:
        - Smooth
        - COnsistent, almost no noise
        - Red, Yellow, Black

Sound:
    - Switch itself
    - Heavy the switch itself is
    - Mechanism of switch
    - Case material (plastic, aluminum, brass, combo)
    - Design(hwo much empty space)
    - Floating keycap design or not
    - Keycaps(ABS, PBT)
    - thickness of keycap.
    - shape of keycap
    - Lubed or not
    - Stabilizers present? Are they modded or not?


* **mel-spectrograms** as a method of feature extraction for a DL mode

* Augmentation:
    * Signals were time-shifted randomly by up to 40% in either direction.
        - Helps increase amount of data
    * Mel-spectrograms generated using:
        * 64 mel brands
        * Window length of 1024 samples
        * hop length of 500 (255 for macbook keystrokes, given their shorter length)
        - results in 64x64 images
    * After Mel-spectrograms, complete masking:
        * take random 10% of both time and frequency axis and setting all values within those ranges to the mean of the spectrogram.
            - 'blocking out' a portion of the image.
            - Called SpecAugment and was foudn to encourage the model to generalise and avoid overfitting.
    